4.1 Parse Training data and find wordTag count, unigram count and bigram count

In [57]:
from os import listdir,getcwd
from os.path import isfile, join
taggedText = []
text = []
filepath = join(getcwd(), "browncopy")
allFiles = [f for f in listdir(filepath) if not (f.startswith('.')) and isfile(join(filepath, f))]
for file in allFiles:
    filename = join(filepath, file)
    for line in open(filename,'r'):
        if line != '\n':
            text.append(line.strip())
#             start tag is <s> and end tag is <e>
            taggedText.append('<s> ' + (line.strip()) + ' <e>')

In [58]:
import collections
wordTagDict = collections.Counter()
unigramTagDict = collections.Counter()
bigramTagDict = collections.Counter()
for line in taggedText:
    words=line.split(' ')
    for wordIndex in range(1,len(words)):
        previousWordTag = words[wordIndex-1]
        currentWordTag = words[wordIndex]
        previousWordTagArray = previousWordTag.split('/')
        currentWordTagArray = currentWordTag.split('/')
        previousWordTagArrayLength = len(previousWordTagArray)
        currentWordTagArrayLength = len(currentWordTagArray)            
        previousTag = previousWordTagArray[-1]
        currentTag = currentWordTagArray[-1]
#         any two words in between the line
        if previousWordTagArrayLength > 1 and currentWordTagArrayLength >1:
            wordTagDict[currentWordTag] +=1
            unigramTagDict[currentTag] += 1
            bigramTagDict[previousTag + ' ' + currentTag ] += 1
#        previous word is start of line
        elif previousWordTagArrayLength == 1 and currentWordTagArrayLength>1:
            wordTagDict[currentWordTag] +=1
            unigramTagDict[currentTag] += 1
            unigramTagDict[previousWordTag] += 1
            bigramTagDict[ previousWordTag + ' ' + currentTag ] += 1
#         current word is end of line
        elif previousWordTagArrayLength > 1 and currentWordTagArrayLength == 1:
            bigramTagDict[previousTag + ' ' + currentWordTag ] += 1 

Printing wordTag count, unigram count and bigram count output to text file

In [9]:
wordTagText = ''
for wordTag in wordTagDict:
    wordTagText += wordTag + ' : ' + str(wordTagDict[wordTag]) + '\n'
unigramTagText = ''
for unigramTag in unigramTagDict:
    unigramTagText += unigramTag + ' : ' + str(unigramTagDict[unigramTag]) + '\n'
bigramTagText = ''
for bigramTag in bigramTagDict:
    bigramTagText += bigramTag + ' : ' + str(bigramTagDict[bigramTag]) + '\n'

In [13]:
# Commented out this code to avoid appending values again
# f = open('wordtagCounts.txt','a')
# f.write(wordTagText)
# f.close()
# f = open('unigramtagCounts.txt','a')
# f.write(unigramTagText)
# f.close()
# f = open('bigramtagCounts.txt','a')
# f.write(bigramTagText)
# f.close()

4.2 Calculate transition probablity

In [59]:
transitionProb = {}
K=0.1
for tagPair in bigramTagDict:
    prevTag = tagPair.split(' ')[0]
    transitionProb[tagPair] = (bigramTagDict[tagPair]+K)/(unigramTagDict[prevTag] + (K*len(bigramTagDict)))

4.3 Calculate emission probablity

In [60]:
emissionProb = {}
K = 0.1
for wordTag in wordTagDict:
    tag = wordTag.split('/')[-1]
    emissionProb[wordTag] = (wordTagDict[wordTag] + K)/(unigramTagDict[tag] + (K * len(wordTagDict)))

Output transition and emission probablity to text file

In [16]:
transitionText = ''
for transitionTagPair in transitionProb:
    transitionText += transitionTagPair + ' : ' + str(transitionProb[transitionTagPair]) + '\n'

In [17]:
emissionText = ''
for emissionPair in emissionProb:
    emissionText += emissionPair + ' : ' + str(emissionProb[emissionPair]) + '\n'

In [18]:
# Commented out this code to avoid appending values again
# f = open('transitionProb.txt','a')
# f.write(transitionText)
# f.close()
# f = open('emissionProb.txt','a')
# f.write(emissionText)
# f.close()

4.4 Generate sentences using HMM algorithm

In [20]:
import numpy as np
for i in range(0,5):
    firstTag = '<s>'
    secondTag = ''
    sentenceProb = 1
    sentence = firstTag + ' '
    while True:
        tagPairList = []
        wordTagList = []
        wordTagProbList = []
        tagPairProbList = []
        wordTagGenDict = {}
        tagPairGenDict = {}
        for tagPair in bigramTagDict:
            if firstTag in tagPair:
                tagPairGenDict[tagPair] = bigramTagDict[tagPair]
        totalTagPairs = sum(tagPairGenDict.values())
        for tagPair in tagPairGenDict:
            tagPairGenDict[tagPair] = tagPairGenDict[tagPair]/totalTagPairs
            tagPairList.append(tagPair)
            tagPairProbList.append(tagPairGenDict[tagPair])
            
        curTagPair = np.random.choice(tagPairList, p=tagPairProbList)
        secondTag = curTagPair.split(' ')[1]
        if secondTag == '<e>':
            sentence += secondTag
            break;
        for wordTag in wordTagDict:
            if secondTag in wordTag:
                wordTagGenDict[wordTag] = wordTagDict[wordTag]
        totalWordTags = sum(wordTagGenDict.values())
        
        for wordTag in wordTagGenDict:
            wordTagGenDict[wordTag] = wordTagGenDict[wordTag]/totalWordTags
            wordTagList.append(wordTag)
            wordTagProbList.append(wordTagGenDict[wordTag])
        curWordTag = np.random.choice(wordTagList, p=wordTagProbList)
        sentenceProb *= transitionProb[curTagPair] * emissionProb[curWordTag]
        sentence += curWordTag + ' '
        firstTag = secondTag
    print('\n')
    print(sentence + ' ' + str(sentenceProb))



<s> I/ppss represents/vbz hall/nn-hl Need/nn-hl of/in-hl Norms/nns-hl meteorites/nns-hl in/in-hl in/in-hl an/at-hl A/at-hl The/at-hl the/at-hl the/at-hl dream/nn-hl warfare/nn-hl co-optation/nn-hl iodine/nn-hl off/rp-hl <e> 1.0812655701071951e-85


<s> Quint/np limitations/nns intervals/nns ,/, I/ppss you/ppss sent/vbd his/pp$ his/pp$ poems/nns effect/nn measurements/nns article/nn and/cc &/cc-tl Government/nn-tl Tumor/nn-tl Island/nn-tl President/nn-tl ./. <e> 1.070531057195176e-62


<s> 2/cd-hl <e> 3.853427882184323e-05


<s> a/at appreciated/vbd Santa/np Manchester/np-tl Chicago/np-tl Figure/nn-tl Morris/np-tl Sherman's/np$ expressivness/nn or/cc elementary/jj question/nn what/wdt-hl which/wdt He/pps Did/dod did/dod did/dod did/dod didn't/dod* did/dod didn't/dod* didn't/dod* written/vbn a/at mat/nn start/nn must/md a/at-hl An/at-hl the/at-hl method/nn-hl chemical/nn-hl water/nn-hl device/nn-hl Life/nn-hl of/in-hl of/in-hl in/in-hl of/in-hl the/at-hl portrayed/vbn-hl ./.-hl ?/.-hl 

4.5 Using viterbi algorithm tag each word in humor.txt file

In [61]:
testFileName = 'humor.txt'
testFile =  open(testFileName,'r')
testText = testFile.read()

In [62]:
updatedText = ''
for ch in range(0,len(testText)):
    updatedText += testText[ch]
    if testText[ch] == '.':
        updatedText += '\n'
    if ch > 2 and (testText[ch] == '!' and testText[ch-2] == '!') or (testText[ch] == '?' and testText[ch-2] == '?' and
                                                                      testText[ch+2] != ')'):
        updatedText += '\n'

In [63]:
testSentences = updatedText.split('\n')

In [19]:
testSentences[0]

'It was among these that Hinkle identified a photograph of Barco ! !'

In [86]:
def findTagsPerSentenceUsingList(sentence):
    index = sentence.split()
    columns = list(unigramTagDict.keys())
    colCount, rowCount = len(columns), len(index)
    df = [[0 for x in range(colCount)] for y in range(rowCount)]
    bp = [[0 for x in range(colCount)] for y in range(rowCount)]
    transitionVocab = len(bigramTagDict)
    emissionVocab = len(wordTagDict)

    for col in columns: 
        word = index[0]
#         smoothing which returns a basic value in case of unknown words
        basicTransitionProb = K / (unigramTagDict[col] + (K * transitionVocab))
        basicEmissionProb = K / (wordTagDict[word] + (K * emissionVocab))
        df[0][columns.index(col)] = transitionProb.get('<s> ' + col, basicTransitionProb) * emissionProb.get(word+'/'+col,basicEmissionProb)
    for i in range(1,len(index)):
        maxVal = 0.0
        maxarg = 0
        word = index[i]
        for col in columns:
            for colIndex in range(0,len(columns)):
                basicTransitionProb = K / (unigramTagDict[colIndex] + (K * transitionVocab))
                basicEmissionProb = K / (wordTagDict[word] + (K * emissionVocab))
                previousTag = df[i-1][colIndex]
                transProb = transitionProb.get(columns[colIndex] + ' ' + col, basicTransitionProb)
                emisProb = emissionProb.get(word +'/'+col, basicEmissionProb)
                checkMaxVal = previousTag * transProb * emisProb
                if checkMaxVal > maxVal:
                    maxVal = checkMaxVal
                    maxarg = colIndex
            df[i][columns.index(col)] = maxVal
            bp[i][columns.index(col)] = maxarg
    return df,bp, index, columns

In [87]:
def generateTaggedSentenceUsingList(df,bp, index, columns,i):
    tagDict = {}
    taggedSentence = ' <EOS> \n'
    firstSentence = '<sentence ID='+ str(i+1) +'> \n' 
    indx = index
    maxVal = 0.0
    maxArg = 0
    lastWord = indx[-1]
    lastWordIndex = len(indx)-1
    for colIndex in range(0,len(columns)):
        checkMaxVal = df[lastWordIndex][colIndex]                  
        if checkMaxVal > maxVal:
            maxVal = checkMaxVal
            maxArg = colIndex
    lastArg = maxArg
    taggedSentence = lastWord + ', ' + columns[maxArg] + '\n' + taggedSentence
    indexList = list(range(0,len(indx),1))
    for j in indexList[::-1]:
        if j is not len(indx)-1:
            curArg = bp[j+1][lastArg]
            sentence = indx[j] + ', ' + columns[curArg] + ' \n '
            taggedSentence = sentence + taggedSentence
            lastArg = curArg
    taggedSentence = firstSentence + taggedSentence
    return taggedSentence

In [93]:
#  viterbi algorithm
for i in range(0,len(testSentences)):
    if testSentences[i] != '':
        df,bp, index, columns = findTagsPerSentenceUsingList(testSentences[i])
        taggedSentence = generateTaggedSentenceUsingList(df,bp, index, columns, i)
        f = open('viterbiOutput.txt','a')
        f.write(taggedSentence)
        f.close()